# GTD2 notebook

## 1. Adat és módszer

**Feladatunk a terrorizmus tulajdonságainak kontintens- és országfüggőségének vizualizálása, valamint a terrorcselekmények klasszifikációja az áldozatok száma alapján.**

GTD2 adatkészlet esetén terrorizmus az a cselekmény, melyre az alábbi pontok közül legalább kettő teljesül:
1. Az erőszakos cselekedet célja egy politikai, gazdasági, vallási vagy társadalmi cél elérése volt.
2. Az erőszakos cselekedet során egyértelmű volt a szándékos kényszerítés, megfélemlítés, vagy valamely más üzenet közvetítése egy nagyobb közönségnek, nem csak az áldozatoknak.
3. Az erőszakos cselekedet során sérültek a nemzetközileg elismert emberi jogok.

Az adatok 1970 és 2016 között történt terrorcselekményeket tartalmazzák. 1970-től 1997-ig rendszeresen frissítették, utána 2007-ig viszont visszamenőleg vitték fel az ismert támadásokat. Ezáltal valószínűleg ez az időszak nem adja vissza a teljes képet. 2007 után újra bekerültek az aktuális történések.

A forrás részletes információkat tartalmaz az
* esemény idejéről, hosszáról
* incidensről (definíció szerinti értelmezése alapján)
* incidens helyéről
* támadásról (típusa, sikeressége)
* fegyverről (típusa, altípusa)
* célpontról/áldozatról (típusa, specifikussága, nemzeti jellege)
* elkövetőről (csoport, létszám, bizonyosság) 
* veszteségekről és következményekről (halálos áldozatok száma, sérültek száma, anyagi kár, túszok)
* egyéb információkról és forrásokról

---

## 2. EDA

### 2.1 Adatok beolvasása

Spark python könyvtár importálása, majd a spark környezet `sc` létrehozása lokál módban.

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext('local', 'gtd2')
sqlc = SQLContext(sc)

Az adatokat `gtd.txt` tabulátorral elválasztott szöveges fájlból olvassuk ki. Majd egy `map` segítségével tagoljuk a tabulátor alapján az oszlopokat.

In [ ]:
raw = sc.textFile("gtd.txt")
data = raw.map(lambda x: x.split('\t'))

# Segéd függvény, amely az adatnak az oszlop indexét adja vissza az oszlopnév alapján
def getIndexByKey(key):
    return data.take(1)[0].index(key)

### 2.2 Terrorcselekmények számának év szerinti összesítése

In [ ]:
import pandas as pd

iyear = getIndexByKey('iyear')
rdd_years = data.map(lambda x: (x[iyear], 1))\
                .reduceByKey(lambda a, b: a+b)\
                .filter(lambda x: x[0]!='iyear')\
                .sortByKey(1)

years = rdd_years.collect()
df_years = pd.DataFrame.from_records(years, columns = ('year', 'count')).apply(pd.to_numeric)

A `df_years` tartalmazza az egyes években összesített terrorcselekmények számát. A `df_years_kill` változóban pedig az összes haláleset található évekre lebontva.

In [ ]:
nkill = getIndexByKey('nkill')
rdd_years_kills = data.map(lambda x: (x[iyear], x[nkill]))\
                      .filter(lambda x: x[0]!='iyear')\
                      .sortByKey(1)
        
df_years_kill = pd.DataFrame.from_records(rdd_years_kills.collect(), columns = ('year', 'killed')).apply(pd.to_numeric)
df_years_kill = df_years_kill.groupby('year')\
                             .sum()\
                             .reset_index()

Statisztikai adatok számításához lesz rá szükség. TODO

In [ ]:
from pyspark.mllib.stat import Statistics
import numpy as np
summary_year = Statistics.colStats(rdd_years)

In [ ]:
print('46 év alatt átlagosan ennyi haláleset történt: ', df_years_kill['killed'].mean())
print(int(df_years_kill.max()['year']),'-ban/-ben történt a legtöbb (',int(df_years_kill.max()['killed']),') haláleset')

### 2.3 Terrorcselekmények az országokban
#### 2.3.1 Incidensek száma az országokban
Az `incidents_in_countries`-ban találhatóak az összes terrorcselekmény száma országokra bontva. Az első öt legtöbb terrorcselekményt tartalmazó ország megjelenítése.

In [ ]:
country_txt = getIndexByKey('country_txt')
incidents_in_countries = data.map(lambda x: (x[country_txt], 1)).reduceByKey(lambda a,b: a+b)
incidents_in_countries = incidents_in_countries.map(lambda x: (x[1], x[0])).sortByKey(0)
incidents_in_countries.take(5)

#### 2.3.2 Halálesetek száma az egyes országokban

In [ ]:
import pandas as pd

country = getIndexByKey('country_txt')
nkill = getIndexByKey('nkill')

rdd_country_kill = data.map(lambda x: (x[country], x[nkill]))\
                       .filter(lambda x: x[0]!='country_txt')\
                       .sortByKey(1)

country_kill = rdd_country_kill.collect()
df_country_kill = pd.DataFrame.from_records(country_kill, columns = ('country', 'killed'))
df_country_kill['killed'] = df_country_kill['killed'].apply(pd.to_numeric)
df_country_kill = df_country_kill.groupby('country').sum().reset_index()
df_country_kill

#### 2.3.3 Halálesetek száma régiókra vetítve

In [ ]:
import pandas as pd

region = getIndexByKey('region')
nkill = getIndexByKey('nkill')
rdd_region_kill = data.map(lambda x: (x[region], x[nkill]))\
                      .filter(lambda x: x[0]!='region')\
                      .sortByKey(1)

region_kill = rdd_region_kill.collect()
df_region_kill = pd.DataFrame.from_records(region_kill, columns = ('region', 'killed')).apply(pd.to_numeric)
#df_region_kill['killed'] = df_region_kill['killed'].apply(pd.to_numeric)
df_region_kill = df_region_kill.groupby('region').sum().reset_index()
df_region_kill

---

## 3. Vizualizációk

### 3.1 Terrorcselekmények időbeli trendje
Az alábbi vizualizációs blokk egy egyszerű idősor diagram, amely ábrázolja, hogy egy évben mennyi terrorcselekmény történt, és az évek során hány halálos áldozat volt.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

m, b = np.polyfit(df_years['year'], df_years['count'], 1)

fig = plt.figure()

pl1 = fig.add_subplot(211)
pl1.set_ylabel('Terrorcselekmények száma')
pl1.set_title('Halálesetek és terrorcselekmények száma 1970-2016')
plt.yscale('linear')
pl1.plot(df_years['year'], df_years['count'])
pl1.plot(df_years['year'], m*df_years['year']+b, '-')

pl2= fig.add_subplot(212)
pl2.set_ylabel('Halálesetek száma')
pl2.set_xlabel('years')
plt.yscale('log')
pl2.plot(df_years_kill['year'], df_years_kill['killed'])

plt.show()

 ### 3.2 Terrortámadások denzitása

Az alábbi vizualizációs blokk az összes terrorcselekményt ábrázolja a hosszúsági és szélességi adatok alapján.
A vizualizációhoz elsősorban a *datashader* könyvtárat használjuk. A codebook alapján, a hosszúsági és szélességi adatok WGS84-es formátumba vannak tárolva, viszont ahhoz, hogy megfelelően tudjuk ábrázolni a pontokat, WebMercator formátumba kell  projektálni a meglévő hosszúsági és szélességi adatokat. Ehhez a *pyproj* nevű könyvtárat használjuk. Ehhez egy-egy  függvényt (*toWebMLon* és *toWebMLat*) definiáltunk, ami megvalósítja a megfelelő projekciókat. A pozíciókat a *pandas* könyvtár segítségével egy dataframe-be tároljuk, amit feltudunk használni a vizualizációhoz. Az ábrán fekete alapra egy 
"hőtérkép"-hez hasonló eredményt várunk, vagyis adott térségben lévő színfoltok fogják jellemezni a terror cselekmények helybeli sűrűségét.

Később a *datashader* könyvtárat arra fogjuk használni, hogy kontinensekre, ill. országokra vetítve ábrázoljuk a terrorcselekmények számát. Minél több terrortámadás történt egy kontinensen vagy egy országban, az annál világosabb színt kap.

In [ ]:
from pyproj import Proj, transform
import pandas as pd

# long/lat. adatok Web mercator formátumba konvertálásához
def toWebMLon(lon):
    loc = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), lon, 0)
    return loc[0]

def toWebMLat(lat):
    loc = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), 0, lat)
    return loc[1]

# indexek
longitude = getIndexByKey('longitude')
latitude = getIndexByKey('latitude')
successful = getIndexByKey('success')
attack_type = getIndexByKey('attacktype1')
weapon_type = getIndexByKey('weaptype1_txt')

points = data.map(lambda x: (x[longitude], x[latitude], x[successful], x[attack_type], x[weapon_type]))\
             .filter(lambda x: x[0]!='longitude')\
             .collect()

locations = pd.DataFrame.from_records(points, columns = ['longitude', 'latitude', 'success', 'attack', 'weapon']).replace({',': '.'}, regex=True)
locations['longitude'] = locations['longitude'].apply(pd.to_numeric)
locations['latitude'] = locations['latitude'].apply(pd.to_numeric)

locations['longitude'] = locations['longitude'].apply(toWebMLon)
locations['latitude'] = locations['latitude'].apply(toWebMLat)

In [ ]:
import datashader as ds
import datashader.glyphs
import datashader.transfer_functions as tf
from functools import partial
from datashader.utils import export_image
from matplotlib.cm import hot

background = "black"
export = partial(export_image, background = background, export_path="export")

x_range = (locations['longitude'].min(), locations['longitude'].max())
y_range = (locations['latitude'].min(), locations['latitude'].max())

plot_width  = int(1500) # Minél nagyobb, annál jobb a felbontás, akár országon belüli eloszlás vizsgálatához
plot_height = int(plot_width*7.0/12)

# Terrorcselekmények denzitása
cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height, x_range=x_range, y_range=y_range)
agg = cvs.points(locations, 'longitude', 'latitude') 
export(tf.shade(agg, cmap = hot, how='eq_hist'), "gtd_on_map")

In [ ]:
if background == "black":
      color_key = {'1':'red', '0':'green'}
else: color_key = {'1':'red', '0':'green'}

cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height, x_range=x_range, y_range=y_range)
agg = cvs.points(locations, 'longitude', 'latitude', ds.count_cat('weapon'))

export(tf.shade(agg, color_key=color_key, how='eq_hist'), "successful attacks")

### 3.3 Terrorcselekmények kontines és országfüggősége

A terrorcselekmények számát régiókra összesítettük a `df_region_kill` dataframe-ben.

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

from matplotlib import pyplot as plt

Z = linkage(df_region_kill, 'average')

plt.title('Régiók klaszterezése')
plt.xlabel('Régiók')
plt.ylabel('Távolság')
dendrogram(Z, leaf_rotation=90)
plt.show()

c, coph_dists = cophenet(Z, pdist(df_region_kill))

---

## 4. Klasszifikáció  

Klasszifikálnunk kell az incidenseket az áldozatok száma alapján. Alkothatnánk például 4 osztályt a percentilisek alapján. Ne nekünk kelljen megmondani, hogy 10 alatt kevésnek számít, 100 felett meg soknak. Beszéljenek a számok! Ha pl az adatkészlet alapján az esetek 25%-ában kevesebben haltak meg 20-nál, 20 legyen a legkevesebb áldozatot jelölő osztály.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

nkill_column = data.map(lambda x: [x[nkill]]).filter(lambda x: x[0]!='nkill' and x[0]!='')
df = pd.DataFrame(nkill_column.collect(),columns=['nkill']).apply(pd.to_numeric)
df.quantile([0.25,0.5,0.75,1])

Vajon hogyan alakul az áldozatok száma az összes incidens esetén?

In [ ]:
nkill_sum = data.map(lambda x: (x[nkill], 1)).filter(lambda x: x[0]!='nkill' and x[0]!='').reduceByKey(lambda a,b: a+b).sortByKey(1)
df2 = pd.DataFrame(nkill_sum.collect(), columns=['nkills','sum']).apply(pd.to_numeric)

plt.plot(df2['nkills'], df2['sum'], '.')
plt.yscale('log')
plt.show()

In [ ]:
plt.bar(df2['nkills'], df2['sum'])
plt.yscale('log')
plt.show()

In [ ]:
def getClassLabel(number):
    borders = (1,5,10,100) # i. oszályba tartozik egy incidens, ha a tömb i. indexű eleménél kevesebb áldozata volt
    for i in range(len(borders)):
        if int(number)<borders[i]:
            return i
    return len(borders)

nkill_classes = data.map(lambda x: x[nkill]).filter(lambda x: x!='nkill' and x!='').map(lambda x: (getClassLabel(x), 1)).reduceByKey(lambda a,b: a+b).sortByKey(1)
nkill_classes.collect()

Alább egy demó látható a döntési fás tanításból.

In [ ]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

# a tulajdonságok
iyear = getIndexByKey('iyear')
imonth = getIndexByKey('imonth')
iday = getIndexByKey('iday')
extended = getIndexByKey('extended')
country = getIndexByKey('country')
region = getIndexByKey('region')
specificity = getIndexByKey('specificity')
vicinity = getIndexByKey('vicinity')
crit1 = getIndexByKey('crit1')
crit2 = getIndexByKey('crit2')
crit3 = getIndexByKey('crit3')
doubtterr = getIndexByKey('doubtterr')
alternative = getIndexByKey('alternative')
multiple = getIndexByKey('multiple')
success = getIndexByKey('success')
suicide = getIndexByKey('suicide')
attacktype1 = getIndexByKey('attacktype1')
attacktype2 = getIndexByKey('attacktype2')
attacktype3 = getIndexByKey('attacktype3')
targtype1 = getIndexByKey('targtype1')
targsubtype1 = getIndexByKey('targsubtype1')
natlty1 = getIndexByKey('natlty1')
targtype2 = getIndexByKey('targtype2')
targsubtype2 = getIndexByKey('targsubtype2')
natlty2 = getIndexByKey('natlty2')
targtype3 = getIndexByKey('targtype3')
targsubtype3 = getIndexByKey('targsubtype3')
natlty3 = getIndexByKey('natlty3')
guncertain1 = getIndexByKey('guncertain1')
guncertain2 = getIndexByKey('guncertain2')
guncertain3 = getIndexByKey('guncertain3')
individual = getIndexByKey('individual')
nperps = getIndexByKey('nperps')
claimed = getIndexByKey('claimed')
claimmode = getIndexByKey('claimmode')
claim2 = getIndexByKey('claim2')
claimmode2 = getIndexByKey('claimmode2')
claim3 = getIndexByKey('claim3')
claimmode3 = getIndexByKey('claimmode3')
compclaim = getIndexByKey('compclaim')
weaptype1 = getIndexByKey('weaptype1')
weapsubtype1 = getIndexByKey('weapsubtype1')
weaptype2 = getIndexByKey('weaptype2')
weapsubtype2 = getIndexByKey('weapsubtype2')
weaptype3 = getIndexByKey('weaptype3')
weapsubtype3 = getIndexByKey('weapsubtype3')
weaptype4 = getIndexByKey('weaptype4')
weapsubtype4 = getIndexByKey('weapsubtype4')
nwound = getIndexByKey('nwound')
nwoundte = getIndexByKey('nwoundte')
proper = getIndexByKey('property')
propextent = getIndexByKey('propextent')
ishostkid = getIndexByKey('ishostkid')
nhostkid = getIndexByKey('nhostkid')
ndays = getIndexByKey('ndays')
ransom = getIndexByKey('ransom')
hostkidoutcome = getIndexByKey('hostkidoutcome')
nreleased = getIndexByKey('nreleased')
INT_LOG = getIndexByKey('INT_LOG')
INT_IDEO = getIndexByKey('INT_IDEO')
INT_MISC = getIndexByKey('INT_MISC')
INT_ANY = getIndexByKey('INT_ANY')

# kezdeti szűrés
data_filtered = data.map(lambda x: np.array(x))\
                    .filter(lambda x: x[nkill]!='nkill' and x[nkill]!='' and x[iyear]!='' and x[imonth]!=''
                            and x[iday]!='' and x[extended]!='' and x[country]!='' and x[region]!='' 
                            and x[specificity]!='' and x[vicinity]!='' and x[crit1]!='' and x[crit2]!=''
                            and x[crit3]!='' and x[doubtterr]!='' and x[multiple]!='' and x[success]!='' 
                            and x[suicide]!='' and x[attacktype1]!='' and x[targtype1]!='' and x[natlty1]!='' 
                            and x[guncertain1]!='' and x[individual]!='' and x[weaptype1]!='' and x[proper]!='' 
                            and x[ishostkid]!='' and x[ransom]!='')

# az új adathalmazunk, címkékkel együtt, tanításhoz (LabeledPoint)
data_rdd_of_labeledpoints = data_filtered.map(lambda x: LabeledPoint(getClassLabel(x[nkill]), x[[iyear, imonth, iday, extended, country,
                                              region, specificity, vicinity, crit1, crit2, crit3, doubtterr, multiple, 
                                              success, suicide, attacktype1, targtype1, natlty1, guncertain1, individual, 
                                              weaptype1, proper, ishostkid, ransom, INT_LOG, INT_IDEO, INT_MISC, INT_ANY]]))

# a PCA-hoz az adathalmaz, vektorokként
data_of_vectors = data_rdd_of_labeledpoints.map(lambda x: x.features)


PCA

In [ ]:
from pyspark.mllib.linalg.distributed import RowMatrix

mat = RowMatrix(data_of_vectors)
# Compute the top 4 principal components.
# Principal components are stored in a local dense matrix.
pc = mat.computePrincipalComponents(4)

# Project the rows to the linear space spanned by the top 4 principal components.
projected = mat.multiply(pc).rows.collect()
pc.toArray()

A döntési fa tanítása

In [ ]:
(trainingData, testData) = data_rdd_of_labeledpoints.randomSplit([0.8, 0.2])

# 3:2, 9:2, 10:2, 11:2, 14:2, 15:2

model = DecisionTree.trainClassifier(trainingData, numClasses=5, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)

In [ ]:
# Modell értékelése, pontosság, hibaarány számítása
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())